In [5]:
import becquerel as bq
from becquerel import Spectrum
from becquerel.tools.isotope import Isotope
from becquerel.tools.isotope_qty import IsotopeQuantity, NeutronIrradiation
import datetime
import numpy as np
import scipy.integrate as integrate
import math as m
import importlib
import pandas as pd
import sys
import os
import csv
import matplotlib.pyplot as plt
import re
sys.path.insert(0,r'C:\Users\ahanks\DoseNet\radwatch-analysis')

In [6]:
import PF
import naa_isotope_analyzer as naa
import analysis_methods as am
import ROI
importlib.reload(PF)
importlib.reload(naa)
importlib.reload(am)
importlib.reload(ROI)

ModuleNotFoundError: No module named 'PF'

In [ ]:
def get_isotopes(energies):
    hl_cut = 600 #in becq.fetch decay radiation
    br_cut =4.0
    df = naa.naa_isotope_analyzer(energies,hl_cut,br_cut,deltae=5.0)
    
    return df

In [ ]:
sample_masses = [1.0,1.1,1.0,1.1,1.0,1.0,1.1,1.0,0.9,1.0]
sample_fish = ["Petrale Sole 1","Halibut","Tilapia","Rockfish","Rock Cod","Catfish","Petrale Sole 2",
               "Wild King Salmon","Swordfish","Oysters"]
sample_mass_unc = 0.1

In [ ]:
sample_index = 1

In [ ]:
#use the ecal and bgspec from google drive
#implement choosing between relevant ecal and bg for diff sample groups
path = r"C:\Users\ahanks\Google Drive\RadWatch\Gamma\Gamma Spectra\Fish Samples 2019"
e_cal = np.loadtxt(path + r"\NAA\11-19-2019-Calibration.Ent")
bgspec =  Spectrum.from_file(path + r"\NAA\Background_Spectrum_12_20_19.Spe")

In [ ]:
#specs for testing
sample_name = r"\NAA\ucb19"
if sample_index < 9:
    sample_name = sample_name + "0"
sample_name = sample_name + str(sample_index+1)
print(sample_name)
spec_30 = Spectrum.from_file(path+sample_name + "-30min.Spe")
spec_03 = Spectrum.from_file(path+sample_name + "-3hr.Spe")
spec_24 = Spectrum.from_file(path+sample_name+"-24hr.Spe")

In [ ]:
specs= [spec_30,spec_03,spec_24]
cal_specs = []
for i in range(len(specs)):
    specs[i].calibrate_like(bgspec)
    cal_specs.append(specs[i])

In [ ]:
energies = []
for i in range(len(cal_specs)):
    energies.append(am.get_energies(cal_specs[i])) #peak finding method, bq.peakfinder


In [ ]:
print(energies)

In [ ]:
energy_shift = 3

In [ ]:
iso_dfs =[]
#isotope analyzer improvement
for e in energies:
    e = e + energy_shift
    iso_dfs.append(get_isotopes(e))

In [ ]:
print(energies)
iso_dfs[2]

In [ ]:
eff_func = am.Efficiency()
eff_func.set_parameters()

In [ ]:
counts = []

uncertainties =[]

icounts = []
iuncertainties = []
#uses PF.PF 
for i in range(len(cal_specs)):
    icounts, iuncertainties = am.get_counts(cal_specs[i],bgspec,energies[i])
    counts.append(icounts)
    uncertainties.append(np.sqrt(icounts))

roi = ROI.ROI(cal_specs[0], bgspec, energies[0], sub_type = 0)
delta_e0 = 5
window0 = np.array([[-2, -1], [-0.5, 0.5], [1, 2]])
roi.set_sideband(delta_e0,window0)
roi.find_peak_energies()
target_peaks = roi.target_peaks
roi_result0,roi_uncer0 = roi.get_counts()
#peakfit = PF.PF(spectrum,background,source_energies)
#pf_result = peakfit.get_counts()
print(target_peaks)
print('ROI result:',roi_result0)
print('uncertainties:',roi_uncer0)

In [ ]:
print(energies[0])
print(counts[0])
print(uncertainties[0])# the uncertainties are all over the place or equal to the real value
print(energies[1])
print(counts[1])
print(uncertainties[1])# the uncertainties are all over the place or equal to the real value


In [ ]:
ec = np.array([[ene, counts[i]/eff_func.get_eff(ene),uncertainties[i]/eff_func.get_eff(ene)] for (i, ene) in enumerate(energies)])

In [ ]:
print("These are for spectrum 1")
print("the energies are",ec[0,0])
print("the true counts are",ec[0,1])
print("the unc on the true counts are",ec[0,2])

In [ ]:
print(ec[1][0])
print(iso_dfs[1].energies[0])

In [ ]:
spectra = 0
for s in range(3):
    iso_dfs[s]['counts'] = "0"
    iso_dfs[s]['count_unc'] = "0"

    for j, e in enumerate(iso_dfs[s]['energies']):
        iso_dfs[s]['counts'][j] = np.zeros(len(e))
        iso_dfs[s]['count_unc'][j] = np.zeros(len(e))

    for i, det_ene in enumerate(ec[s][0]):
        for j, em_enes in enumerate(iso_dfs[s]['energies']):
            for k, in_ene in enumerate(em_enes):
                if (det_ene+energy_shift) == in_ene:
                    iso_dfs[s]['counts'][j][k] = round(ec[s][1][i])
                    iso_dfs[s]['count_unc'][j][k] = round(ec[s][2][i])

In [ ]:
for i,df in enumerate(iso_dfs):
    df['cps'] = df['counts']/cal_specs[i].livetime
    df['cps_unc'] = df['count_unc']/cal_specs[i].livetime
    df = df.drop('counts',1)
    iso_dfs[i] = df.drop('count_unc',1)
iso_dfs[0]

In [ ]:
iso_dfs[1]

In [ ]:
#iso_dfs[1] = iso_dfs[1].drop([4])
#iso_dfs[1]

In [ ]:
iso_dfs[2]

In [ ]:
iso_dfs[2] = iso_dfs[2].drop([0,1,2,3,4,8])
iso_dfs[2]

In [ ]:
for df in iso_dfs:
    Activities = []
    act_unc = []
    for i in range(len(df['cps'])):
        Activities.append(np.sum(df['cps'].iloc[i])/np.sum(df['branching_ratios'].iloc[i]))
        act_unc.append(np.sqrt(np.sum(np.power(df['cps_unc'].iloc[i],2)))/np.sum(df['branching_ratios'].iloc[i]))
    df['Activity'] = Activities
    df['Act_unc'] = act_unc

In [ ]:
iso_dfs[0] #a list of activities and activity uncertainties in df

In [ ]:
t0 = cal_specs[0].start_time.timestamp() - (4*24*60*60) #time of irrad
print(t0)

In [ ]:
flux = 2*10**11 #from davis trigger reactor
Tirrad = 1 *3600 #sec irradiated
irr_start = '2019-11-08 09:14:00'
irr_stop = '2019-11-08 10:14:00'

In [ ]:
new_dfs = am.calculate_concentration(iso_dfs,'Activity','mass(g)',flux,irr_start,irr_stop,cal_specs)

In [ ]:
new_dfs = am.calculate_concentration(iso_dfs,'Act_unc','mass(g)-unc',flux,irr_start,irr_stop,cal_specs)

In [ ]:
new_dfs[0]

In [ ]:
for df in new_dfs:
    df['ppm'] = df['mass(g)']/sample_masses[sample_index]*10000
    df['ppm-unc'] = df['mass(g)-unc']/sample_masses[sample_index]*10000
new_dfs[0]

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=am.get_initial_isotopes(new_dfs[0]['isotopes']), y=new_dfs[0]['ppm'],
    error_y=dict(type='data', array=new_dfs[0]['ppm-unc']),
    name="30 min"
))
fig.add_trace(go.Bar(
    x=am.get_initial_isotopes(new_dfs[1]['isotopes']), y=new_dfs[1]['ppm'],
    error_y=dict(type='data', array=new_dfs[1]['ppm-unc']),
    name="3 hr"
))
fig.add_trace(go.Bar(
    x=am.get_initial_isotopes(new_dfs[2]['isotopes']), y=new_dfs[2]['ppm'],
    error_y=dict(type='data', array=new_dfs[2]['ppm-unc']),
    name="24 hr"
))
fig.update_layout(barmode='group',
                title={
                    'text': sample_fish[sample_index],
#                    'y':0.9,
                    'x':0.5,
                    'xanchor': 'center'},
#                    'yanchor': 'top'},
                xaxis_title="Isotope",
                yaxis_title="Concentration (ppm)"
#                font=dict(
#                    family="Courier New, monospace",
#                    size=18,
#                    color="RebeccaPurple"
#                )
)
fig.update_yaxes(type="log")
fig.show()